In [10]:
import selenium
from selenium import webdriver
import urllib.request
import ssl

In [11]:
import time
# 相簿效期只有3個月，須判斷是否仍在效期內。（上傳日期：2018-07-29）

localtime = time.localtime(time.time())
# time.time() : seconds since 12:00am, January 1, 1970
# time.localtime() : show local current time in struct_time
# time.asctime() : show time in "Www Mmm dd hh:mm:ss yyyy" formate
urlInvalidDate = time.strptime("2018-10-28 23:59:59", "%Y-%m-%d %H:%M:%S")

In [12]:
# 圖片網址可能所在之 xpath & attribute 對應
xpath_attribute = {
    '//div[@data-swiper-slide-index]':'style',
    '//div[@data-swiper-slide-index]/img':'src'
}

# urls to be get
url_dict = {}

In [13]:
driver = webdriver.Chrome(r'D:\pyLadies\2018_0805_nmnsCrawler\chromedriver.exe')
totalDays = 3

In [14]:
# method : get the clean URLs, and put in url_dict{}
def findURL(xpath,attribute):
    images = driver.find_elements_by_xpath(xpath)
    index = 0 #圖片張數計算

    for image in images:
        tagContent = image.get_attribute(attribute)
        
        if tagContent.find('\"') != -1: tagContent = tagContent.split('\"')[-1]
            # ↑split the string if it comes from attribute "style"
        else: pass

        if tagContent.find("https://") != -1:
            index += 1
            url = tagContent.split('\"')[0]
            # ↑split to get clear url from attribute "style"
            if url not in url_dict:
                url_dict[url] = "Day" + str(day) + "_" + str(index).zfill(3) + ".jpg"
            else:
                pass
        else:
            return False

In [15]:
# main
if localtime > urlInvalidDate: # check the date, future/now is larger than past
    print("相簿已逾期！請手動確認檔案是否仍存在。")
else:
    for day in range(1,totalDays+1): # get the url of the day
        urlDay = 'http://www.nmns.edu.tw/web/learn/feature/photo.htm?id={idNo}'.format(idNo=12+day)
             # ↑url of day 1 = http://www.nmns.edu.tw/web/learn/feature/photo.htm?id=13
        driver.get(urlDay)

        for xpath,attribute in  xpath_attribute.items(): # get url from xpath
            if findURL(xpath,attribute) != False:
                print("Day",day,"取得網址！","目前字典長度:",len(url_dict))
            else:
                print( "注意：「", xpath, " + ", attribute, "」的組合失敗。" )
                  
    for key,value in url_dict.items(): #download images from url,and save in file name
#        print(key,value)
        ssl._create_default_https_context = ssl._create_unverified_context
            # ↑ to pass the SSL certificate verify
        resource = urllib.request.urlopen(key)
        output = open(value,"wb") # write binary
        output.write(resource.read())
        output.close()
    print("下載結束")

注意：「 //div[@data-swiper-slide-index]  +  style 」的組合失敗。
Day 1 取得網址！ 目前字典長度: 85
注意：「 //div[@data-swiper-slide-index]  +  style 」的組合失敗。
Day 2 取得網址！ 目前字典長度: 363
注意：「 //div[@data-swiper-slide-index]  +  style 」的組合失敗。
Day 3 取得網址！ 目前字典長度: 475
下載結束


In [16]:
driver.close()